# xCDAT vs. CDAT Spatial Averaging Output `dtype`

Objective:

* Figure out the root cause for the large floating point differences (absolute and relative), which might be related to the `dtype`.

Questions:
* Is xCDAT or CDAT doing something incorrectly?
* Does CDAT cast the type of the data from `float32` to `float64`?
   * If it does, when does it cast the type?

Resources:
https://discourse.pangeo.io/t/variable-type-changing-when-using-xarray/1823/2

* Same array, same bits, but different precision. In general, arbitrary smooth decimals can’t be represented exactly using float32 precision.
* The index type (Float64Index) actually comes from Pandas (pandas.Float64Index — pandas 1.3.3 documentation). In this case, promoting a float32 type to a float64 in the context of indexing should not affect selection operations, since float32 can safely be cast to float64
* The number 10.45 is interpreted differently if it is float32 vs float64.
* Builtin python floats are 64-bit, so IMO Xarray does the right thing by promoting everything to float64 when making comparisons with your lat values.


In [2]:
import numpy as np

import xcdat
import cdms2
import cdutil

In [3]:
fn = "/p/user_pub/climate_work/pochedley1/surface/gistemp1200_GHCNv4_ERSSTv5.nc"

ds_xcdat = xcdat.open_dataset(fn)
ds_cdat = cdms2.open(fn)

### 1. Check dtype for the variable `"tempanomaly"`

In [3]:

ds_xcdat["tempanomaly"].dtype

dtype('float32')

In [4]:

ds_cdat("tempanomaly").dtype

dtype('float32')

#### The `dtype` of the variable is `float32` for both libraries, so they agree.

### 2. Check dtype for the temporal averaging output for `"tempanomaly"`

xCDAT

In [5]:
ds_xcdat_avg = ds_xcdat.temporal.climatology("tempanomaly", freq="month")
ta_xcdat_avg = ds_xcdat_avg["tempanomaly"]


In [6]:
ta_xcdat_avg

<xarray.DataArray 'tempanomaly' (time: 12, lat: 90, lon: 180)>
array([[[ 0.03216783,  0.03216783,  0.03216783, ...,  0.03216783,
          0.03216783,  0.03216783],
        [ 0.03216783,  0.03216783,  0.03216783, ...,  0.03216783,
          0.03216783,  0.03216783],
        [ 0.03216783,  0.03216783,  0.03216783, ...,  0.03216783,
          0.03216783,  0.03216783],
        ...,
        [ 0.92027974,  0.92027974,  0.92027974, ...,  0.92027974,
          0.92027974,  0.92027974],
        [ 0.92027974,  0.92027974,  0.92027974, ...,  0.92027974,
          0.92027974,  0.92027974],
        [ 0.92027974,  0.92027974,  0.92027974, ...,  0.92027974,
          0.92027974,  0.92027974]],

       [[-0.11165388, -0.11165388, -0.11165388, ..., -0.11165388,
         -0.11165388, -0.11165388],
        [-0.11165388, -0.11165388, -0.11165388, ..., -0.11165388,
         -0.11165388, -0.11165388],
        [-0.11165388, -0.11165388, -0.11165388, ..., -0.11165388,
         -0.11165388, -0.11165388],
...
        [ 0.8019719 ,  0.8019719 ,  0.8019719 , ...,  0.8019719 ,
          0.8019719 ,  0.8019719 ],
        [ 0.8019719 ,  0.8019719 ,  0.8019719 , ...,  0.8019719 ,
          0.8019719 ,  0.8019719 ],
        [ 0.8019719 ,  0.8019719 ,  0.8019719 , ...,  0.8019719 ,
          0.8019719 ,  0.8019719 ]],

       [[ 0.13211268,  0.13211268,  0.13211268, ...,  0.13211268,
          0.13211268,  0.13211268],
        [ 0.13211268,  0.13211268,  0.13211268, ...,  0.13211268,
          0.13211268,  0.13211268],
        [ 0.13211268,  0.13211268,  0.13211268, ...,  0.13211268,
          0.13211268,  0.13211268],
        ...,
        [ 0.6506337 ,  0.6506337 ,  0.6506337 , ...,  0.6506337 ,
          0.6506337 ,  0.6506337 ],
        [ 0.6506337 ,  0.6506337 ,  0.6506337 , ...,  0.6506337 ,
          0.6506337 ,  0.6506337 ],
        [ 0.6506337 ,  0.6506337 ,  0.6506337 , ...,  0.6506337 ,
          0.6506337 ,  0.6506337 ]]], dtype=float32)
Coordinates:
  * lat      (lat) float32 -89.0 -87.0 -85.0 -83.0 -81.0 ... 83.0 85.0 87.0 89.0
  * lon      (lon) float32 -179.0 -177.0 -175.0 -173.0 ... 175.0 177.0 179.0
  * time     (time) object 0001-01-01 00:00:00 ... 0001-12-01 00:00:00
Attributes:
    operation:  temporal_avg
    mode:       climatology
    freq:       month
    weighted:   True

In [7]:
ta_xcdat_avg.dtype

dtype('float32')

CDAT

In [8]:
ta_cdat_avg = cdutil.ANNUALCYCLE(ds_cdat("tempanomaly"))

In [9]:
ta_cdat_avg

tempanomaly
masked_array(
  data=[[[--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         ...,
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --]],

        [[--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         ...,
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --]],

        [[--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         ...,
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --]],

        ...,

        [[-0.019999999552965164, -0.019999999552965164,
          -0.019999999552965164, ..., -0.019999999552965164,
          -0.019999999552965164, -0.019999999552965164],
         [-0.01999

In [10]:
ta_cdat_avg.dtype

dtype('float64')

#### xCDAT is returning `float32` (same as input dtype).
#### CDAT is returning `float64` (not the same as input type).

### 3. Why is CDAT returning `float64`?

If you do a search for float in [genutil.averager](https://github.com/CDAT/genutil/blob/master/Lib/averager.py), you will find `numpy.float` referenced several times.

### 4. What exactly is a `numpy.float`? It is `float64`.

In [11]:
np.float

/tmp/ipykernel_46796/699376334.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.float


float

## Key Takeaways

* `ds.spatial.average()` maintains a `dtype` of `float32`, which is **CORRECT**.
* `cdutil.averager()`/`genutil.averager()` is typecasting the `dtype` to `float64`, which is **INCORRECT**.
* Floating point comparisons between CDAT and xCDAT results in large differences since the `dtype` of the outputs are different (`float32` vs. `float64`)

**Based on these findings, we are confident that xarray/xCDAT is doing the right thing with floating points, while CDAT is not.**